Try using tf converter to create tflite model

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [4]:
m0 = Dense(units=1, input_shape=[1])
model_m1 = Sequential([m0])
model_m1.compile(optimizer='sgd', loss='mean_squared_error')

xs = np.array([-1.0, 0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)
ys = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0], dtype=float)

model_m1.fit(xs,ys,epochs=500)

print(model_m1.predict([10.0]))

Epoch 1/500
1/1 [==============================] - 1s 1s/step - loss: 33.0453
Epoch 2/500
1/1 [==============================] - 0s 2ms/step - loss: 26.3245
Epoch 3/500
1/1 [==============================] - 0s 2ms/step - loss: 21.0302
Epoch 4/500
1/1 [==============================] - 0s 2ms/step - loss: 16.8584
Epoch 5/500
1/1 [==============================] - 0s 3ms/step - loss: 13.5697
Epoch 6/500
1/1 [==============================] - 0s 2ms/step - loss: 10.9761
Epoch 7/500
1/1 [==============================] - 0s 2ms/step - loss: 8.9294
Epoch 8/500
1/1 [==============================] - 0s 3ms/step - loss: 7.3131
Epoch 9/500
1/1 [==============================] - 0s 3ms/step - loss: 6.0356
Epoch 10/500
1/1 [==============================] - 0s 2ms/step - loss: 5.0247
Epoch 11/500
1/1 [==============================] - 0s 2ms/step - loss: 4.2237
Epoch 12/500
1/1 [==============================] - 0s 2ms/step - loss: 3.5879
Epoch 13/500
1/1 [==============================] - 0s 5

In [6]:
export_dir = 'm1'
tf.saved_model.save(model_m1,export_dir)

INFO:tensorflow:Assets written to: m1\assets


In [7]:
# Convert model to tflite
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
tflite_model = converter.convert()

In [8]:
# Setup the 'edge' environment
import pathlib
tflite_model_file = pathlib.Path('tf_model.tflite')
tflite_model_file.write_bytes(tflite_model)



904

In [9]:
# load tflite model and assign tensors
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

# Get input and output tensors
input = interpreter.get_input_details()
output = interpreter.get_output_details()
print(input, output)

[{'name': 'serving_default_dense_input:0', 'index': 0, 'shape': array([1, 1]), 'shape_signature': array([-1,  1]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}] [{'name': 'StatefulPartitionedCall:0', 'index': 3, 'shape': array([1, 1]), 'shape_signature': array([-1,  1]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [10]:
# Inference only
to_predict = np.array([[20.0]], dtype=np.float32)
print(to_predict)

interpreter.set_tensor(input[0]['index'], to_predict)
interpreter.invoke()

tflite_results = interpreter.get_tensor(output[0]['index'])
print(tflite_results)

[[20.]]
[[38.947594]]
